<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 02 - Data Manipulation ##

**Table of Contents**
<br>
This notebook explores the fundamentals of data acquisition and manipulation using DataFrame APIs, covering essential techniques for handling and processing datasets. This notebook covers the below sections: 
1. [Data Background](#Data-Background)
1. [cuDF and pandas](#cuDF-and-pandas)
    * [pandas](#pandas)
    * [cuDF](#cuDF)
3. [Data Acquisition](#Data-Acquisition)
4. [Initial Data Exploration](#Initial-Data-Exploration)
5. [Indexing and Data Selection with `.loc` Accessor](#Indexing-and-Data-Selection-with-.loc-Accessor)
6. [Basic Operations](#Basic-Operations)
    * [Exercise #1 - Convert `county` Column to Title Case](#Exercise-#1---Convert-county-Column-to-Title-Case)
7. [Aggregation](#Aggregation)
8. [Applying User-Defined Functions (UDFs) with `.map()` and `.apply()`](#Applying-User-Defined-Functions-(UDFs)-with-.map()-and-.apply())
9. [Filtering with `.loc` and Boolean Mask](#Filtering-with-.loc-and-Boolean-Mask)
    * [Exercise #2 - Counties North of Sunderland](#Exercise-#2---Counties-North-of-Sunderland)
10. [Creating New Columns](#Creating-New-Columns)
11. [pandas vs. cuDF](#pandas-vs.-cuDF)
12. [cuDF pandas](#cuDF-pandas)
    * [Exercise #3 - Automatic Acceleration](#Exercise-#3---Automatic-Acceleration)

## Data Background ##
For this workshop, we will be reading almost 60 million records (corresponding to the entire population of England and Wales) which were synthesized from official UK census data. 

## cuDF and pandas ##

### pandas ###
[pandas](https://pandas.pydata.org/) is a widely-used open-source library for data manipulation and analysis in Python. It provides high-performance, easy-to-use data structures and tools for working with structured data. It popularized the term DataFrame as a data structure for statistical computing. In data science, pandas is used for: 
* **Data loading and writing**: reads from and writes to various file formats like CSV, Excel, JSON, and SQL databases
* **Data cleaning and processing/preprocessing**: helps users with handling missing data, merging datasets, and reshaping data
* **Data analysis**: performs grouping, aggregating, and statistical operations

**Note**: Data preprocessing refers to the process of transforming raw data into a format that is more suitable for analysis and other downstream tasks. 

### cuDF ###
Similarly, [cuDF](https://docs.rapids.ai/api/cudf/stable/) is a Python GPU DataFrame library for loading, joining, aggregating, filtering, and otherwise manipulating data. cuDF is designed to accelerate data science workflows by utilizing the parallel processing power of GPUs, potentially offering significant speed improvements over CPU-based alternatives for large datasets. The key features of cuDF include: 
* **GPU Acceleration**: leverages NVIDIA GPUs for fast data processing and analysis
* **pandas-like API**: provides users a familiar interface and transition to GPU-based computing
* **Integration with other RAPIDS libraries**: works seamlessly with other GPU-accelerated tools in the RAPIDS ecosystem

**Note**: Both Pandas and cuDF serve similar purposes in data manipulation and analysis, but cuDF is specifically optimized for GPU acceleration, making it particularly useful for working with large datasets where performance is critical.

## Data Acquisition ##
In our context, data acquisition refers to the process of collecting and importing data from various sources into a Python environment for analysis, processing, and manipulation. Data can come from a variety of sources: 
* Local file in various formats
* Databases
* APIs
* Web scraping

It's worth noting that Python's rich ecosystem of libraries makes it versatile for acquiring data from various sources, allowing data scientists to work with diverse datasets efficiently. CPU processing will be responsible for acquiring data from APIs or Web Scraping. In most cases, network bandwidth will likely be the bottleneck. Furthermore, cuDF doesn't have a way to get transactions from SQL data bases directly into GPU memory. The recommended approach for reading data from a database is to first use CPU-based methods (i.e. pandas), then convert to cuDF for GPU-accelerated processing.  

Below we use the `head` linux command to display the beginning of the data file. This allows us to understand how to read the data correctly. 

In [6]:
# DO NOT CHANGE THIS CELL
!head -n 5 data/uk_pop.csv

"age","sex","county","lat","long","name"
0,"m","DARLINGTON",54.53364379,-1.524400639,"FRANCIS"
0,"m","DARLINGTON",54.42625551,-1.465313919,"EDWARD"
0,"m","DARLINGTON",54.55520036,-1.496417277,"TEDDY"
0,"m","DARLINGTON",54.54790635,-1.572341399,"ANGUS"


One row will represent one person. We have information about their `age`, `sex`, `county`, location, and `name`. Using cuDF, the RAPIDS API providing a GPU-accelerated DataFrame, we can read data from [a variety of formats](https://rapidsai.github.io/projects/cudf/en/0.10.0/api.html#module-cudf.io.csv), including csv, json, parquet, feather, orc, and pandas DataFrames, among others. 

In [7]:
# DO NOT CHANGE THIS CELL
import cudf
import cupy as cp
import numpy as np

from datetime import datetime
import random
import time

Below we read the data from a local csv file directly into GPU memory with the `read_csv()` function. 

In [8]:
# DO NOT CHANGE THIS CELL
start=time.time()
df=cudf.read_csv('./data/uk_pop.csv')
print(f'Duration: {round(time.time()-start, 2)} seconds')

Duration: 1.76 seconds


**Note**: Because of the sophisticated GPU memory management behind the scenes in cuDF, the first data load into a fresh RAPIDS memory environment is sometimes substantially slower than subsequent loads. The [RAPIDS Memory Manager](https://github.com/rapidsai/rmm) is preparing additional memory to accommodate the array of data science operations that we may be interested in using on the data, rather than allocating and deallocating the memory repeatedly throughout the workflow. 

Below we get the general information about the DataFrame with the `DataFrame.info()` method. 

In [9]:
# DO NOT CHANGE THIS CELL
df.info(memory_usage='deep')

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 58479894 entries, 0 to 58479893
Data columns (total 6 columns):
 #   Column  Dtype
---  ------  -----
 0   age     int64
 1   sex     object
 2   county  object
 3   lat     float64
 4   long    float64
 5   name    object
dtypes: float64(2), int64(1), object(3)
memory usage: 2.9 GB


The **DataFrame** is a two-dimensional labeled data structure. It's organized in rows and columns, similar to a spreadsheet or SQL table. Both rows and columns have labels. Rows are typically labeled with an index, while columns have column names. Data is aligned based on row and column labels when performing operations. This is useful for enabling highly efficient vectorized operations across columns or rows. A **Series** refers to a one-dimensional array and is typically associated with a single column of data with an index. 

There are ~60MM records across 6 columns. cuDF is able to read data from local files directly into the GPU very efficiently. By default, cuDF samples the dataset to infer the most appropriate data types for each columns. 

**Note**: The DataFrame has `.dtypes` and `.columns` attributes that can be used to get similar information. 

## Initial Data Exploration ##
Now that we have some data loaded, let's do some initial exploration. 

Below we preview the DataFrame with the `DataFrame.head()` method. 

In [10]:
# DO NOT CHANGE THIS CELL
df.head()

,age,sex,county,lat,long,name
0,0,m,DARLINGTON,54.533644,-1.524401,FRANCIS
1,0,m,DARLINGTON,54.426256,-1.465314,EDWARD
2,0,m,DARLINGTON,54.555200,-1.496417,TEDDY
3,0,m,DARLINGTON,54.547906,-1.572341,ANGUS
4,0,m,DARLINGTON,54.477639,-1.605995,CHARLIE


## Indexing and Data Selection with `.loc` Accessor ##
The `.loc` accessor in cuDF DataFrames is used for label-based indexing and selection of data. It allows us to access and manipulate data in a DataFrame based on row and column labels. We can use `DataFrame.loc[row_label(s), column_label(s)]` to access a group of rows and columns. When selecting multiple labels, a list (`[]`) is used. Furthermore, we can use the slicing operator (`:`, i.e. `start:end`) to specify a range of elements. 

In [11]:
# DO NOT CHANGE THIS CELL
# get first cell
display(df.loc[0, 'age'])
print('-'*40)

# get multiple rows and columns
display(df.loc[[0, 1, 2], ['age', 'sex', 'county']])
print('-'*40)

# slice a range of rows and columns
display(df.loc[0:5, 'age':'county'])
print('-'*40)

# slice a range of rows and columns
display(df.loc[:10, :'name'])

0

----------------------------------------


,age,sex,county
0,0,m,DARLINGTON
1,0,m,DARLINGTON
2,0,m,DARLINGTON


----------------------------------------


,age,sex,county
0,0,m,DARLINGTON
1,0,m,DARLINGTON
2,0,m,DARLINGTON
3,0,m,DARLINGTON
4,0,m,DARLINGTON
5,0,m,DARLINGTON


----------------------------------------


,age,sex,county,lat,long,name
0,0,m,DARLINGTON,54.533644,-1.524401,FRANCIS
1,0,m,DARLINGTON,54.426256,-1.465314,EDWARD
2,0,m,DARLINGTON,54.555200,-1.496417,TEDDY
3,0,m,DARLINGTON,54.547906,-1.572341,ANGUS
4,0,m,DARLINGTON,54.477639,-1.605995,CHARLIE
5,0,m,DARLINGTON,54.522900,-1.599255,VICTOR
6,0,m,DARLINGTON,54.501872,-1.667874,EAMONN
7,0,m,DARLINGTON,54.554709,-1.494506,HARRY
8,0,m,DARLINGTON,54.602288,-1.586457,HECTOR
9,0,m,DARLINGTON,54.489992,-1.652537,THEODORE


**Note**: `df[column_label(s)]` is another way to access specific columns, similar to `df.loc[:, column_labels]`. 

## Basic Operations ##
cuDF support a wide range of operations for numerical data. Although strings are not a data type traditionally associated with GPUs, cuDF supports powerful accelerated string operations.
* Numerical operations:
    * Arithmetic operations: addition, subtraction, multiplication, division
* String operations:
    * Case conversion: `.upper()`, `.lower()`, `.title()`
    * String manipulation: concatenation, substring, extraction, padding
    * Pattern matching: `contains()`
    * Splitting: `.split()`
* Comparison operations: greater than, less than, equal to, etc.

These operations will be performed element-wise for each row. This allows for efficient **vectorized operations** across entire columns. These operations are implemented as vector operations instead of iteration because vector operations can be applied to entire arrays of data, instead of iterating through each element individually. Vectorization is significantly faster than iterating over elements, especially for large datasets. When operating on multiple columns, operations are aligned by index, ensuring that calculations are performed on the correct corresponding elements across columns. These element-wise operations are typically highly optimized and can be much faster than explicit loops, especially for large datasets. We can get the underlying array of data with the `.values` attribute. This is useful when we want to perform operations on the underlying data. 

**Note**: Iterating over a cuDF Series, DataFrame or Index is not supported. This is because iterating over data that resides on the GPU will yield extremely poor performance, as GPUs are optimized for highly parallel operations rather than sequential operations. 

Below we calculate the birth year for each person. 

In [12]:
# DO NOT CHANGE THIS CELL
# get current year
current_year=datetime.now().year

# derive the birth year
display(current_year-df.loc[:, 'age'])

# get the age array (CuPy for cuDF)
age_ary=df.loc[:, 'age'].values

# derive the birth year
current_year-age_ary

0           2025
1           2025
2           2025
3           2025
4           2025
            ... 
58479889    1935
58479890    1935
58479891    1935
58479892    1935
58479893    1935
Name: age, Length: 58479894, dtype: int64

array([2025, 2025, 2025, ..., 1935, 1935, 1935])

When performing operations between a DataFrame and a scalar value, the scalar is "broadcast" to match the shape of the DataFrame, effectively applying it to each element. 

```
current_year - df.loc[:, 'age']
-------------------------------
  (scalar)          (array)    
    2024,    -         0
    2024,    -         0
    2024,    -         0
    2024,    -         0
    2024,    -         0
    ...      -         ...
```

This partially explains why cuDF provides significant performance improvements over pandas, especially for large datasets. The parallel processing architecture of GPUs are designed with thousands of small, specialized cores that can execute many operations simultaneously. This architecture is ideal for vectorized operations, which perform the same instruction on multiple data elements in parallel. 

<a name='s4.1'></a>
### Exercise #1 - Convert `county` Column to Title Case ###
As it stands, all of the counties are UPPERCASE. We want to convert the `county` column to title case. 

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell to convert the `county` column to title case. 

In [13]:
df['county'].str.<<<<FIXME>>>>

SyntaxError: invalid syntax (1846494622.py, line 1)

Click ... for solution. 

Performing comparison operations or applying conditions create boolean values (`True`/`False`) that corresponds element-wise. 

Below we check if each person is an adult. 

In [14]:
# DO NOT CHANGE THIS CELL
df['age']>=18

0           False
1           False
2           False
3           False
4           False
            ...  
58479889     True
58479890     True
58479891     True
58479892     True
58479893     True
Name: age, Length: 58479894, dtype: bool

## Aggregation ##
Aggregation is an important operation for data science tasks, allowing us to summarize and analyze grouped data. It's commonly used for tasks like calculating totals, averages, counts, etc. cuDF supports common aggregations like `.sum()`, `.mean()`, `.min()`, `.max()`, `.count()`, `.std()`(standard deviation), etc. It also supports more advanced aggregations like `.quantile()` and `.corr()` (correlation). With the `axis` parameter, aggregation operations can be applied column-wise (`0`) or row-wise (`1`). 

When the aggregation is implemented as a vector operation, specifically a reduction operation, it is very efficient on the GPU becasue a large number of data elements can be processed simultaneously and in parallel. Column-wise operations also benefit from the [Apache Arrow columnar memory format](https://arrow.apache.org/docs/format/Columnar.html). 

<p><img src='images/sum_reduction.png' width=720></p>

Below we calculate the arithmetic mean of `lat` and `long` to get an approximate center. 

In [15]:
# DO NOT CHANGE THIS CELL
df[['lat', 'long']].mean()

lat     52.350600
long    -1.304956
dtype: float64

## Applying User-Defined Functions (UDFs) with `.map()` and `.apply()` ##
The `.map()` and `.apply()` methods are the primary ways of applying user-defined functions element-wise, and row or column-wise, respectively. We can pass a callable function (built-in or user-defined) as the argument, which is then applied to the entire data structure. Not all operations can be vectorized, especially complex custom logic. In such cases, methods like `.apply()` or custom UDFs might be necessary.

Below we use `.apply()` to check if each person is an adult. 

In [16]:
# DO NOT CHNAGE THIS CELL
# define a function to check if age is greater than or equal to 18
start=time.time()
def is_adult(row): 
    if row['age']>=18: 
        return 1
    else: 
        return 0

# derive the birth year
display(df.apply(is_adult, axis=1))
print(f'Duration: {round(time.time()-start, 2)} seconds')

0           0
1           0
2           0
3           0
4           0
           ..
58479889    1
58479890    1
58479891    1
58479892    1
58479893    1
Length: 58479894, dtype: int64

Duration: 2.23 seconds


We can also use a [**lambda function**](https://docs.python.org/3/glossary.html#term-lambda) when the function is simple. Lambda functions are limited to a single expression but can include a conditional statement and mulitple arguments. 

In [17]:
# DO NOT CHANGE THIS CELL
# derive the birth year
start=time.time()
display(df.apply(lambda x: 1 if x['age']>=18 else 0, axis=1))
print(f'Duration: {round(time.time()-start, 2)} seconds')

0           0
1           0
2           0
3           0
4           0
           ..
58479889    1
58479890    1
58479891    1
58479892    1
58479893    1
Length: 58479894, dtype: int64

Duration: 0.05 seconds


**Note**: The `.apply()` function in pandas accepts any user-defined function that can include arbitrary operations that are applied to each value of a Series and DataFrame. cuDF also supports `.apply()`, but it relies on Numba to JIT compile the UDF (not in scope) and execute it on the GPU. This can be extremely fast, but imposes a few limitations on what operations are allowed in the UDF. See the docs on [UDFs](https://docs.rapids.ai/api/cudf/stable/user_guide/guide-to-udfs/) for details.

In [18]:
# DO NOT CHANGE THIS CELL
# derive the birth year
start=time.time()
display((df['age']>=18).astype('int'))
print(f'Duration: {round(time.time()-start, 2)} seconds')

0           0
1           0
2           0
3           0
4           0
           ..
58479889    1
58479890    1
58479891    1
58479892    1
58479893    1
Name: age, Length: 58479894, dtype: int64

Duration: 0.02 seconds


Below we use `Series.map()` to determine the number of characters in each person's name. 

In [19]:
# DO NOT CHANGE THIS CELL
df['name'].map(lambda x: len(x))

0           7
1           6
2           5
3           5
4           7
           ..
58479889    5
58479890    8
58479891    7
58479892    7
58479893    8
Name: name, Length: 58479894, dtype: int32

## Filtering with `.loc` and Boolean Mask ##
A boolean mask is an array of `True`/`False` values that corresponds element-wise to another array or data structure. It's used for filtering and selecting data based on certain conditions. In this context, the mask can be used to index or filter a DataFrame with `.loc`, selecting only the elements where the mask is `True`. 

**Note**: Boolean masking is often more efficient than iterative approaches, especially for large datasets, as it leverages vectorized operations. 

Below we use the `.loc` accessor and a boolean mask to filter people whose names start with an `E`. 

In [20]:
# DO NOT CHANGE THIS CELL
boolean_mask=df['name'].str.startswith('E')
df.loc[boolean_mask]

,age,sex,county,lat,long,name
1,0,m,DARLINGTON,54.426256,-1.465314,EDWARD
6,0,m,DARLINGTON,54.501872,-1.667874,EAMONN
34,0,m,DARLINGTON,54.483065,-1.501312,ETHAN
45,0,m,DARLINGTON,54.640205,-1.558986,ELVIN
49,0,m,DARLINGTON,54.575450,-1.600592,EDWARD
...,...,...,...,...,...,...
58479859,90,f,NEWPORT,51.576452,-2.891774,EDIE
58479867,90,f,NEWPORT,51.555083,-3.080259,ELEANOR
58479871,90,f,NEWPORT,51.515820,-2.839532,EMERSON
58479875,90,f,NEWPORT,51.510140,-3.004406,ELLA


Multiple conditions can be combined using logical operators (`&` and `|`). 

**Note**: When using multiple conditions, it's important to wrap each condition in parentheses (`(` and `)`) to ensure correct order of operations. 

Below we use the `.loc` accessor and multiple conditions to filter adults whose names start with an `E`. 

In [21]:
# DO NOT CHANGE THIS CELL
df[(df['age']>=18) | (df['name'].str.startswith('E'))]

,age,sex,county,lat,long,name
1,0,m,DARLINGTON,54.426256,-1.465314,EDWARD
6,0,m,DARLINGTON,54.501872,-1.667874,EAMONN
34,0,m,DARLINGTON,54.483065,-1.501312,ETHAN
45,0,m,DARLINGTON,54.640205,-1.558986,ELVIN
49,0,m,DARLINGTON,54.575450,-1.600592,EDWARD
...,...,...,...,...,...,...
58479889,90,f,NEWPORT,51.626744,-2.859381,FREYA
58479890,90,f,NEWPORT,51.546043,-2.897815,GEORGINA
58479891,90,f,NEWPORT,51.605268,-2.849656,REBECCA
58479892,90,f,NEWPORT,51.554649,-2.934364,JESSICA


<a name='s4.1'></a>
### Exercise #2 - Counties North of Sunderland ###
This exercise will require to use the `.loc` accessor, and several of the techniques described above. We want to identify the latitude of the northernmost resident of Sunderland county (the person with the maximum `lat` value), and then determine which counties have any residents north of this resident. Use the `Series.unique()` method of to de-duplicate the result.

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell to identify counties north of Sunderland. 

In [22]:
sunderland_residents=df.loc[<<<<FIXME>>>>]
northmost_sunderland_lat=sunderland_residents['lat'].max()
df.loc[df['lat'] > northmost_sunderland_lat]['county'].unique()

SyntaxError: invalid syntax (1614588699.py, line 1)

Click ... for solution. 

## Creating New Columns ##

We can create new columns by assigning values to the column label. The new column should have the same number of rows as the existing DataFrame. Typically, we create new columns by performing operations on existing columns. 

Below we create a few additional columns. 

In [ ]:
# DO NOT CHANGE THIS CELL
# get current year
current_year=datetime.now().year

# numerical operations
df['birth_year']=current_year-df['age']

# string operations
df['sex_normalize']=df['sex'].str.upper()
df['county_normalize']=df['county'].str.title().str.replace(' ', '_')
df['name']=df['name'].str.title()

# preview
df.head()

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## pandas vs. cuDF ##
Except for being much more performant with large datasets, cuDF looks and feels a lot like Pandas. By way of review, cuDF and pandas share the below similarities: 
* **API similarity**: cuDF provides a pandas-like API that is familiar to data engineers and data scientists. It aims to implement many of the same functions and operations as pandas, allowing users to easily accelerate their existing pandas workflows.
* **Similar operations**: cuDF implements many common pandas operations such as filtering, joining, aggregating, and groupby.

<p><img src='images/pandas_vs_cudf.png' width=1080></p>

Comparing to pandas, cuDF tends to perform better for large datasets because of the follow features: 
* GPUs excel at parallel computation, which is advantageous for many data science and machine learning tasks.
* GPUs typically have much higher memory bandwidth than CPUs, allowing for faster data access in memory-bound operations.
* cuDF leverages GPU's ability to perform vectorized operations efficiently, which is particularly beneficial for large datasets.
* cuDF uses a columnar data format, which can lead to more efficient memory access patterns on GPUs. When performing data operations on cuDF Dataframes, column operations are typically much more performant than row-wise operations.

**Note**: It's important to note that the performance advantage of cuDF over pandas can vary depending on the specific operation, data size, and hardware configuration. For smaller datasets or simpler operations, the overhead of GPU initialization might make pandas on CPU faster.

## cuDF pandas ##
Starting with version `23.10.01`, cuDF introduced a **pandas accelerator mode** (`cudf.pandas`) that supports 100% of the pandas API. This mode allows users to accelerate pandas code on the GPU without requiring any code changes. Not all operations can be performed on the GPU. When using `cudf.pandas`, operations that can be accelerated will run on the GPU, while unsupported operations will automatically fall back to pandas on the CPU. For example, `.read_sql()`. this will first read sql with pandas and move the data to cuDF. 

There are two ways to activate cuDF pandas:
- Jupyter Magic Command
```
%load_ext cudf.pandas
import pandas
...
```
- Python Import
```
import cudf.pandas
cudf.pandas.install()

import pandas as pd
...
```

**Note**: There are no other changes required - this is useful to quickly accelerate existing workloads with minimum code change. More information about cuDF pandas can be found [here](https://docs.rapids.ai/api/cudf/stable/cudf_pandas/). 

cuDF pandas is a no code change accelerator for pandas for automatic acceleration of any supported pandas call. 

Below we run some basic DataFrame operations with pandas, before demonstrating how cudf pandas is enabled. 

In [ ]:
# DO NOT CHANGE THIS CELL
# %load_ext cudf.pandas

In [ ]:
# DO NOT CHANGE THIS CELL
import pandas as pd
import time
from datetime import datetime

In [ ]:
# %%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
start=time.time()

df=pd.read_csv('./data/uk_pop.csv')
current_year=datetime.now().year

df['birth_year']=current_year-df['age']

df['sex_normalize']=df['sex'].str.upper()
df['county_normalize']=df['county'].str.title().str.replace(' ', '_')
df['name']=df['name'].str.title()

print(f'Duration: {round(time.time()-start, 2)} seconds')

display(df.head())

<a name='s4.1'></a>
### Exercise #3 - Automatic Acceleration ###
**Instructions**: <br>
* Go back to the top of this subsection, re-execute the cells and uncomment the `%load_ext` magic command to accelerate with cuDF pandas. 
* Observe the acceleration. 
* Go back to the top of this subsection, re-execute the cells and uncomment the `%%cudf.pandas.line_profile` magic command to use the line profiler. 
* Observe the output from the line profiler. 

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](1-03_memory_management.ipynb). 

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">